In [1]:
import tushare as ts
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing

In [2]:
code_dict = {'飞荣达':'300602','万集科技':'300552','瑞丰光电':'300241','德赛电池':'000049','金溢科技':'002869'}

In [3]:
for i in code_dict.values():
    data = ts.get_k_data(i,start='2017-05-13',end='2019-09-30')
    data.to_csv('select_tech_stock\\'+i+'.csv',index=False)
    print('股票代码',i,' : ',len(data),'条数据')

股票代码 300602  :  584 条数据
股票代码 300552  :  584 条数据
股票代码 300241  :  584 条数据
股票代码 000049  :  584 条数据
股票代码 002869  :  584 条数据


In [4]:
target_stock = pd.read_csv('select_tech_stock\\300602.csv')
target_stock.head()

,date,open,close,high,low,volume,code
0,2017-05-15,23.248,23.637,24.589,23.119,21697.0,300602
1,2017-05-16,23.637,24.365,24.414,23.463,27122.0,300602
2,2017-05-17,25.012,25.431,26.801,24.933,49232.0,300602
3,2017-05-18,24.913,25.710,25.854,24.663,29886.0,300602
4,2017-05-19,25.565,25.391,26.671,25.391,31672.0,300602


### 分割训练集和测试集

In [5]:
def segmentation_train_test(data,alpha=0.8):
    train_length = int(len(data)*alpha)
    train = data[:train_length]
    test = data[train_length:]
    return train,test

### 分割特征及标签

In [6]:
def segmentation_xy(data,observation=30,pred=5,scaler='standard'):
    features = ['open','high','low','close','volume']
    group = observation + pred
    data = pd.DataFrame(data,columns=features)
    if scaler == 'minmax':
        scaler = preprocessing.MinMaxScaler()
    elif scaler =='standard':
        scaler = preprocessing.StandardScaler()
    for i in range(len(list(data.columns))):
        data[features[i]] = scaler.fit_transform(np.reshape(np.array(data[features[i]]),(-1,1)))
    windows,labels = list(),list()
    for i in range(len(data)-group):
        windows.append(np.array(data[i:i+observation]))
        labels.append(1 if data.close[i+group] > data.close[i+observation] else 0)
    windows = np.array(windows)
    labels = np.array(labels)
    return windows,labels

### 分割特征及标签（窗口归一化）

In [16]:
def segmentation_xy_windows(data,observation=30,pred=5,scaler='standard'):
    features = ['open','high','low','close','volume']
    group = observation + pred
    data = pd.DataFrame(data,columns=features)
    if scaler == 'minmax':
        scaler = preprocessing.MinMaxScaler()
    elif scaler =='standard':
        scaler = preprocessing.StandardScaler()
#     for i in range(len(list(data.columns))):
#         data[features[i]] = scaler.fit_transform(np.reshape(np.array(data[features[i]]),(-1,1)))
    windows,labels = list(),list()
    for i in range(len(data)-group):
        temp_data = data[i:i+observation]
#         temp_data = (temp_data - np.mean(temp_data,axis=0))/np.std(temp_data,axis=0,ddof=1)
        temp_data = (temp_data - np.min(temp_data,axis=0))/(np.max(temp_data,axis=0)-np.min(temp_data,axis=0))
        windows.append(np.array(temp_data))
        print(temp_data)
        labels.append(1 if data.close[i+group] > data.close[i+observation] else 0)
    windows = np.array(windows)
    labels = np.array(labels)
    return windows,labels

In [17]:
segmentation_xy_windows(data,observation=30,pred=5,scaler='standard')


        open      high       low     close    volume
0   0.000000  0.000000  0.000000  0.000000  0.000613
1   0.316206  0.133003  0.327848  0.146944  0.000000
2   0.446562  0.279147  0.463004  0.308406  0.000157
3   0.590100  0.440069  0.611826  0.486196  0.000300
4   0.748011  0.617105  0.775551  0.681788  0.001833
5   0.921832  0.811978  0.871538  0.897087  0.353197
6   0.900657  0.788238  0.784197  0.744050  0.902721
7   0.757465  0.873804  0.741085  0.965394  1.000000
8   0.915760  0.820853  0.824122  0.813309  0.820372
9   0.881788  0.912322  0.889549  0.870859  0.811403
10  1.000000  1.000000  1.000000  0.959777  0.975670
11  0.938434  0.978414  0.946368  1.000000  0.805200
12  0.817532  0.838216  0.835120  0.883949  0.717528
13  0.966027  0.949203  0.873092  0.800219  0.638552
14  0.749548  0.677251  0.761485  0.715013  0.490800
15  0.765036  0.752219  0.779097  0.790366  0.485034
16  0.822067  0.701853  0.769295  0.674315  0.468349
17  0.729142  0.639983  0.750090  0.696782  0.

        open      high       low     close    volume
30  0.788546  0.664739  0.640115  0.589772  0.298449
31  0.648499  0.578009  0.647932  0.593464  0.056594
32  0.633439  0.632007  0.633287  0.639712  0.171980
33  0.675750  0.621423  0.663467  0.648758  0.093957
34  0.697879  0.610741  0.670295  0.610726  0.104626
35  0.696855  0.903665  0.688799  0.854796  0.827464
36  0.980945  0.968248  0.971799  1.000000  1.000000
37  1.000000  1.000000  1.000000  0.957353  0.643741
38  0.934535  0.851627  0.784089  0.725007  0.474710
39  0.759348  0.759114  0.709183  0.652451  0.242438
40  0.699928  0.657977  0.603107  0.710514  0.275222
41  0.756275  0.635829  0.660499  0.607957  0.116136
42  0.659666  0.600157  0.617752  0.576202  0.093788
43  0.571048  0.460506  0.223827  0.187852  0.334894
44  0.196394  0.126225  0.074906  0.165144  0.195750
45  0.151009  0.085751  0.100237  0.157851  0.138212
46  0.159103  0.138671  0.126460  0.108834  0.123185
47  0.093638  0.072227  0.111914  0.108834  0.

88  0.550671  0.489164  0.604242  0.543545  0.017298
        open      high       low     close    volume
60  0.321144  0.228218  0.320077  0.246243  0.011644
61  0.235168  0.204334  0.118583  0.105973  0.089842
62  0.030539  0.000000  0.012051  0.000000  0.002280
63  0.000000  0.010836  0.000000  0.117341  0.000000
64  0.103287  0.141309  0.158833  0.225241  0.056766
65  0.179732  0.386997  0.251145  0.405202  0.299117
66  0.365104  0.413092  0.400578  0.426012  0.233009
67  0.372885  0.365104  0.459870  0.401541  0.092167
68  0.388057  0.521672  0.488069  0.577649  0.263404
69  0.527718  0.693498  0.590022  0.490559  0.270444
70  0.472282  0.521672  0.457460  0.518882  0.212895
71  0.321144  0.226006  0.210894  0.185549  0.643916
72  0.118459  0.293454  0.149434  0.373025  0.380624
73  0.460805  0.904246  0.500121  0.784008  0.967192
74  0.718926  0.954224  0.857797  0.748170  1.000000
75  0.671076  0.873950  0.781875  0.643931  0.476902
76  0.657654  0.956435  0.852977  0.928131  0.

117  0.148194  0.116361  0.019086  0.034757  0.022769
         open      high       low     close    volume
89   0.115139  0.182984  0.162154  0.186149  0.025463
90   0.153472  0.192016  0.198550  0.242515  0.033944
91   0.217639  0.293586  0.253884  0.296277  0.170139
92   0.290000  0.656414  0.266903  0.667274  0.609836
93   0.537083  0.554843  0.573754  0.507420  0.281513
94   0.515278  0.485340  0.025448  0.161807  0.357595
95   0.380000  0.580497  0.444296  0.653215  0.226834
96   0.918056  1.000000  0.905459  0.866962  1.000000
97   0.822500  0.900916  0.806480  0.825957  0.607894
98   0.874306  0.966623  0.842876  0.948841  0.530187
99   0.919444  0.960079  1.000000  1.000000  0.395840
100  1.000000  0.890707  0.828377  0.776100  0.295317
101  0.795139  0.812173  0.839917  0.820880  0.152220
102  0.857917  0.770942  0.845835  0.769721  0.122251
103  0.760972  0.665445  0.421068  0.380630  0.300250
104  0.381389  0.382330  0.416778  0.385837  0.050638
105  0.407361  0.406806  0.2

         open      high       low     close    volume
116  0.966824  1.000000  0.943819  0.915542  0.607607
117  0.908092  0.893446  0.823610  0.802182  0.511404
118  0.838377  0.846628  0.831457  0.882141  0.386404
119  0.874916  0.844257  0.842716  0.861111  0.353657
120  0.839386  0.925092  0.890936  0.914417  0.759111
121  0.892625  0.940382  0.944956  0.946581  0.413163
122  0.925801  0.993007  0.929148  0.986617  0.919343
123  1.000000  0.984829  1.000000  1.000000  0.588876
124  0.993387  0.957923  0.902195  0.865497  0.701516
125  0.856086  0.894631  0.874104  0.866622  0.314220
126  0.851603  0.833709  0.826908  0.782164  0.250573
127  0.780767  0.752993  0.508928  0.489991  1.000000
128  0.412015  0.421595  0.231434  0.429937  0.461073
129  0.412015  0.367666  0.330263  0.356613  0.391119
130  0.336696  0.341946  0.313431  0.296671  0.321611
131  0.296794  0.221287  0.199932  0.177688  0.367610
132  0.130688  0.130023  0.167406  0.182186  0.112003
133  0.191661  0.130023  0.1

...

471  0.966608  0.874528  0.850615  0.793017  0.276783
         open      high       low     close    volume
443  0.104569  0.118670  0.100176  0.095594  0.203236
444  0.092267  0.164021  0.120387  0.213633  0.216501
445  0.265378  0.277400  0.227592  0.231920  0.345617
446  0.289982  0.254724  0.222320  0.249377  0.321729
447  0.193322  0.162509  0.080844  0.092269  0.144051
448  0.156415  0.119426  0.096661  0.103076  0.064085
449  0.123023  0.084656  0.107206  0.124688  0.013059
450  0.109842  0.084656  0.120387  0.135495  0.002871
451  0.134446  0.086168  0.089631  0.098088  0.003292
452  0.091388  0.086924  0.112478  0.113882  0.045864
453  0.107206  0.079365  0.103691  0.128845  0.021196
454  0.101054  0.058201  0.098418  0.087282  0.008917
455  0.077329  0.050642  0.000000  0.000000  0.099609
456  0.000000  0.000000  0.023726  0.028263  0.000000
457  0.034271  0.050642  0.054482  0.064007  0.047182
458  0.042179  0.046863  0.041301  0.088113  0.018642
459  0.134446  0.194255  0.1

501  0.665851  0.651882  0.601342  0.612551  0.065109
         open      high       low     close    volume
473  1.000000  1.000000  0.872483  0.901774  0.213292
474  0.822521  0.836022  0.617450  0.620737  0.491191
475  0.653611  0.643817  0.660403  0.684857  0.220513
476  0.701346  0.673387  0.563758  0.556617  0.212474
477  0.571603  0.615591  0.570470  0.638472  0.238367
478  0.962056  0.912634  1.000000  1.000000  0.721944
479  0.632803  0.618280  0.644295  0.583902  0.750055
480  0.235006  0.349462  0.275168  0.208731  0.388265
481  0.265606  0.241935  0.006711  0.040928  0.539720
482  0.000000  0.053763  0.000000  0.053206  0.173572
483  0.020808  0.000000  0.033557  0.000000  0.000000
484  0.095471  0.174731  0.073826  0.242838  0.400425
485  0.326805  0.400538  0.285906  0.366985  0.446468
486  0.379437  0.409946  0.402685  0.388813  0.216832
487  0.418605  0.373656  0.425503  0.439291  0.055403
488  0.449204  0.358871  0.336913  0.312415  0.109942
489  0.375765  0.369624  0.2

534  0.835270  0.782645  0.820367  0.834867  0.121559
         open      high       low     close    volume
506  0.000000  0.000000  0.000000  0.000000  0.235534
507  0.069869  0.103175  0.068340  0.080517  0.227977
508  0.133188  0.093474  0.112041  0.054672  0.161985
509  0.131004  0.104497  0.135286  0.067097  0.025936
510  0.131004  0.067019  0.104603  0.052684  0.000000
511  0.165066  0.080247  0.127383  0.049702  0.028135
512  0.099127  0.070547  0.096234  0.073062  0.050411
513  0.149782  0.093474  0.131567  0.105865  0.072107
514  0.163755  0.216490  0.188285  0.267893  0.363533
515  0.344978  0.339506  0.298001  0.313121  0.658171
516  0.331878  0.253968  0.303115  0.257455  0.209933
517  0.268996  0.329365  0.300790  0.317097  0.356513
518  0.323144  0.397707  0.344956  0.412028  0.442748
519  0.449782  0.515432  0.493259  0.604871  0.243831
520  0.728384  0.703704  0.655044  0.646123  0.920767
521  0.608297  0.602293  0.642027  0.646123  0.436784
522  0.692576  0.645944  0.6

561  0.856720  1.000000  0.927157  1.000000  0.863291
         open      high       low     close    volume
533  0.213641  0.135499  0.066248  0.037296  0.528734
534  0.084253  0.029234  0.000000  0.000000  0.278765
535  0.072217  0.121578  0.058252  0.088578  0.406023
536  0.203109  0.167517  0.203883  0.121212  0.439505
537  0.193079  0.125290  0.166191  0.121678  0.342502
538  0.178034  0.099304  0.121074  0.054545  0.215673
539  0.133400  0.089559  0.098801  0.124476  0.263506
540  0.161484  0.101624  0.151342  0.084848  0.081638
541  0.142929  0.097448  0.137636  0.096503  0.077428
542  0.102808  0.042691  0.040548  0.025641  0.356361
543  0.000000  0.000000  0.009138  0.000932  0.000000
544  0.174524  0.173086  0.185037  0.146853  0.915796
545  0.153460  0.104408  0.103370  0.099301  0.454441
546  0.186058  0.163805  0.174757  0.139394  0.503503
547  0.211133  0.316009  0.169046  0.270396  1.000000
548  0.336008  0.275174  0.320388  0.238228  0.437001
549  0.320963  0.380046  0.3

(array([[[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          6.13132782e-04],
         [3.16206141e-01, 1.33003016e-01, 3.27847950e-01, 1.46944021e-01,
          0.00000000e+00],
         [4.46562392e-01, 2.79146919e-01, 4.63003546e-01, 3.08406321e-01,
          1.56544540e-04],
         ...,
         [6.69459283e-01, 6.05687204e-01, 6.85101805e-01, 6.54655369e-01,
          4.96696258e-01],
         [6.98551170e-01, 5.83239983e-01, 6.90162171e-01, 6.43897563e-01,
          2.92536087e-01],
         [7.04200453e-01, 6.02283498e-01, 7.14467865e-01, 6.20049505e-01,
          2.43792031e-01]],
 
        [[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
          0.00000000e+00],
         [1.90636767e-01, 1.68563335e-01, 2.01078902e-01, 1.89275152e-01,
          1.56544540e-04],
         [4.00550778e-01, 3.54171843e-01, 4.22490960e-01, 3.97689861e-01,
          3.00043702e-04],
         ...,
         [5.59152476e-01, 5.19306266e-01, 5.39036102e-01, 5.

### 堆叠数组

In [7]:
totalx,totaly = np.array(list()),np.array(list())
for i in code_dict.values():
    data = pd.read_csv('select_tech_stock\\'+i+'.csv')
    train,test = segmentation_train_test(data,alpha=0.8)
    train_x,train_y = segmentation_xy(train,30,5,'standard')
#     test_x,test_y = segmentation_xy(test,30,5,'standard')
    if totalx.shape[0] == 0:
        totalx = train_x
    else:
        totalx = np.hstack((totalx,train_x))
    if totaly.shape[0] == 0:
        totaly = train_y
    else:
        pass
print(totalx.shape)
totalx = np.reshape(totalx,(-1,5,30,5,1))
print(totalx.shape,totaly.shape)

(432, 150, 5)
(432, 5, 30, 5, 1) (432,)


In [45]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers

In [49]:
model = keras.Sequential()
model.add(layers.Conv3D(32, kernel_size=(1, 1, 1), input_shape=(totalx.shape[1:]), padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model.add(layers.Activation('softmax'))
model.add(layers.MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model.add(layers.Dropout(0.25))

model.add(layers.Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model.add(layers.Activation('relu'))
model.add(layers.Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model.add(layers.Activation('softmax'))
model.add(layers.MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='sigmoid'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))

model.compile(loss=keras.losses.SparseCategoricalCrossentropy(),
#               loss=losses.categorical_crossentropy,
              optimizer=optimizers.Adam(), metrics=['accuracy'])
model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_36 (Conv3D)           (None, 5, 30, 5, 32)      64        
_________________________________________________________________
activation_33 (Activation)   (None, 5, 30, 5, 32)      0         
_________________________________________________________________
conv3d_37 (Conv3D)           (None, 5, 30, 5, 32)      27680     
_________________________________________________________________
activation_34 (Activation)   (None, 5, 30, 5, 32)      0         
_________________________________________________________________
max_pooling3d_16 (MaxPooling (None, 2, 10, 2, 32)      0         
_________________________________________________________________
dropout_24 (Dropout)         (None, 2, 10, 2, 32)      0         
_________________________________________________________________
conv3d_38 (Conv3D)           (None, 2, 10, 2, 64)    

In [50]:
history = model.fit(totalx, totaly, validation_data=(totalx, totaly), batch_size=128,epochs=100, verbose=1, shuffle=True)

Train on 432 samples, validate on 432 samples
Epoch 1/100
432/432 [==============================] - 3s 7ms/sample - loss: 0.7773 - accuracy: 0.5023 - val_loss: 0.7391 - val_accuracy: 0.5347
Epoch 2/100
432/432 [==============================] - 2s 6ms/sample - loss: 0.8479 - accuracy: 0.4954 - val_loss: 0.6917 - val_accuracy: 0.5347
Epoch 3/100
432/432 [==============================] - 2s 6ms/sample - loss: 0.8534 - accuracy: 0.4560 - val_loss: 0.6994 - val_accuracy: 0.4653
Epoch 4/100
432/432 [==============================] - 2s 6ms/sample - loss: 0.8140 - accuracy: 0.4815 - val_loss: 0.6917 - val_accuracy: 0.5347
Epoch 5/100
432/432 [==============================] - 2s 6ms/sample - loss: 0.7606 - accuracy: 0.5347 - val_loss: 0.6916 - val_accuracy: 0.5347
Epoch 6/100
432/432 [==============================] - 2s 6ms/sample - loss: 0.8303 - accuracy: 0.4884 - val_loss: 0.6914 - val_accuracy: 0.5347
Epoch 7/100
432/432 [==============================] - 2s 6ms/sample - loss: 0.7323 

Epoch 57/100
432/432 [==============================] - 3s 7ms/sample - loss: 0.6374 - accuracy: 0.6435 - val_loss: 0.6128 - val_accuracy: 0.6759
Epoch 58/100
432/432 [==============================] - 3s 7ms/sample - loss: 0.6375 - accuracy: 0.6435 - val_loss: 0.6057 - val_accuracy: 0.6806
Epoch 59/100
432/432 [==============================] - 3s 7ms/sample - loss: 0.6278 - accuracy: 0.6667 - val_loss: 0.5855 - val_accuracy: 0.6898
Epoch 60/100
432/432 [==============================] - 3s 7ms/sample - loss: 0.6129 - accuracy: 0.6690 - val_loss: 0.6139 - val_accuracy: 0.6435
Epoch 61/100
432/432 [==============================] - 3s 7ms/sample - loss: 0.6240 - accuracy: 0.6574 - val_loss: 0.5847 - val_accuracy: 0.6921
Epoch 62/100
432/432 [==============================] - 3s 7ms/sample - loss: 0.6213 - accuracy: 0.6782 - val_loss: 0.5882 - val_accuracy: 0.6991
Epoch 63/100
432/432 [==============================] - 3s 7ms/sample - loss: 0.6050 - accuracy: 0.6574 - val_loss: 0.5910 -

In [52]:
loss, acc = model.evaluate(totalx, totaly, verbose=0)